## CS431/631 Big Data Infrastructure
### Winter 2021 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Yifu Zhang
* **ID:** y2644zha

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [2]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize

data_dic = {}
token_list = []
# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        
        unique_list = []
        for word in t:
            if word in unique_list:
                continue
            else:
                unique_list.append(word)
                
            if word in token_list:
                continue
            else:
                token_list.append(word)
                
        unique_len = len(unique_list)
        for word in unique_list:
            # print(word)
            word_pos = unique_list.index(word)
            pos = 0
            while (pos != word_pos) or (pos <= unique_len-1):
                if pos == word_pos:
                    pos += 1
                    continue
                if pos > unique_len-1:
                    break
                if (unique_list[pos] == unique_list[word_pos]):
                    pos += 1
                    continue
                if (word, unique_list[pos]) in data_dic:
                    data_dic[(word, unique_list[pos])] += 1
                else:
                    data_dic[(word, unique_list[pos])] = 1
                # print(pos)
                pos += 1
                
    print("Number of distinct tokens =", len(token_list))
    print("Number of distinct token pairs =", len(data_dic))

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

Number of distinct tokens = 25975
Number of distinct token pairs = 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [3]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10

line_count = 0
single_dic = {}
data_dic = {}

with open('Shakespeare.txt') as f:
    for line in f:
        line_count += 1
        # tokenize, one line at a time
        t = simple_tokenize(line)
        
        # unique word in this line
        unique_list = []
        for word in t:
            if word in unique_list:
                continue
            else:
                unique_list.append(word)
        # print(unique_list)
        
        #
        unique_len = len(unique_list)
        for word in unique_list:
            # print(word)
            if word in single_dic:
                single_dic[word] += 1
            else:
                single_dic[word] = 1
            
            word_pos = unique_list.index(word)
            pos = 0
            while (pos != word_pos) or (pos <= unique_len-1):
                if pos == word_pos:
                    pos += 1
                    continue
                if pos > unique_len-1:
                    break
                if (unique_list[pos] == unique_list[word_pos]):
                    pos += 1
                    continue
                if (word, unique_list[pos]) in data_dic:
                    data_dic[(word, unique_list[pos])] += 1
                else:
                    data_dic[(word, unique_list[pos])] = 1
                # print(pos)
                pos += 1

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
    
        PMI_dict = {}
    
        for pair in data_dic:
            if (pair[0] == 'fire') and (data_dic[pair] >= threshold):
                PMI_dict[pair] = data_dic[pair]
    
        for key in PMI_dict:
            PMI_dict[key] = log(PMI_dict[key]/single_dic[key[0]]/single_dic[key[1]]*line_count, 10)
        
        PMI_dict = {k: v for k, v in sorted(PMI_dict.items(), key=lambda item: item[1], reverse=True)[0:5]}
        order_keys = list(PMI_dict.keys())
            
        
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        
        print(order_keys[0][1])
        print("  n({0}) = {1}".format(q_tokens[0], single_dic[q_tokens[0]]))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], order_keys[0][1],
                                                                 data_dic[order_keys[0]], PMI_dict[order_keys[0]]))   
        print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], order_keys[1][1],
                                                                 data_dic[order_keys[1]], PMI_dict[order_keys[1]]))
        print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], order_keys[2][1],
                                                                 data_dic[order_keys[2]], PMI_dict[order_keys[2]]))
        print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], order_keys[3][1],
                                                                 data_dic[order_keys[3]], PMI_dict[order_keys[3]]))
        print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], order_keys[4][1],
                                                                 data_dic[order_keys[4]], PMI_dict[order_keys[4]]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        
        nxy = data_dic[(q_tokens[0], q_tokens[1])]
        nx = single_dic[q_tokens[0]]
        ny = single_dic[q_tokens[1]]
        ans = log(nxy/nx/ny*line_count, 10)
        
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1], data_dic[(q_tokens[0], q_tokens[1])]))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1], ans))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")

Input 1 or 2 space-separated tokens (return to quit): fire
Input a positive integer frequency threshold: 12
out
  n(fire) = 259
  high PMI tokens with respect to fire (threshold: 12):
    n(fire,out) = 13,  PMI(fire,out) = 0.644679634005233
    n(fire,on) = 16,  PMI(fire,on) = 0.4053204107137161
    n(fire,as) = 24,  PMI(fire,as) = 0.36319843154380427
    n(fire,with) = 30,  PMI(fire,with) = 0.27374667155200966
    n(fire,the) = 94,  PMI(fire,the) = 0.26220897812374616
Input 1 or 2 space-separated tokens (return to quit): fire peace
  n(fire,peace) = 1
  PMI(fire,peace) = -0.0277431383231655
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

Since the method I used in _Question2_ is pairs, as the data get larger, the running time will significantly increase, which means it will get much slower. 3 times slower by the graph in lecture. However, it will not fail.


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.